In [ ]:
# 패키지 설치
!pip install pickle5 --quiet

In [ ]:
# 패키지 임포트
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle5 as pickle
sns.set()

In [ ]:
# 데이터 로드
with open('oecd_cli.pkl', 'rb') as f:
    oecd_cli_df = pickle.load(f)

with open('spx_price.pkl', 'rb') as f:
    spx_df = pickle.load(f)

In [ ]:
oecd_cli_df.index = pd.to_datetime(oecd_cli_df.index)
spx_df.index = pd.to_datetime(spx_df.index)

In [ ]:
# 선견편향 제거
oecd_cli_df = oecd_cli_df.shift(1)

# 시점별 국가 수 계산
noc = len(oecd_cli_df.columns) - oecd_cli_df.diff().isna().sum(axis=1)

# 확산 지수
diffusion_index = (oecd_cli_df.diff() > 0).sum(axis=1) / noc

In [ ]:
# 확산 지수 그래프
diffusion_index.plot(figsize=(12, 8))

In [ ]:
# 트레이딩 시그널
spx_signal = diffusion_index
spx_signal.index = pd.to_datetime(spx_signal.index)

In [ ]:
# 백테스팅
df = pd.concat([spx_signal, spx_df, diffusion_index], axis=1, join='inner')
df.columns = ['spx_signal', 'spx', 'index']
df['spx_rets'] = df['spx'].pct_change().fillna(0)
df['spx_port_rets'] = df['spx_rets'] * df['spx_signal'].shift() - abs(df['spx_signal'].diff()) * 0.0002

In [ ]:
# 시각화
plt.figure(figsize=(12, 8))

# 지수 및 트레이딩 전략
ax1 = plt.subplot(211)
ax1.plot((1 + df['spx_port_rets'].loc['2000':]).cumprod(), label='Filtered Equity Beta')
ax1.plot((1 + df['spx_rets'].loc['2000':]).cumprod(), label='S&P 500 Index')
plt.legend()

# 확산 지수
ax2 = plt.subplot(212)
ax2.plot(df['index'].loc['2000':], label='Diffusion Index', color='tab:green')
plt.legend()
plt.tight_layout()
plt.show()